In [1]:
import ipywidgets as widgets
from IPython.display import display
import datetime
import geemap
import ee
import json
import os
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# initialize ee 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# import all FMSE py functions 
from FMSE import *


In [2]:
Map = geemap.Map()
# Define widgets
w_case_study = widgets.Dropdown(
    options=[
        ('Select a case study', 'default'),
        ('Shikarpur', 'shikarpur'),
        ('Nhamatanda', 'nhamatanda'),
        ('Ernukulam', 'ernukulam'),
        ('Sylhet', 'sylhet')
    ],
    value='default',
    description='Case Study:',
)

w_startDate = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

w_endDate = widgets.DatePicker(
    description='End Date',
    disabled=False
)

w_preDays = widgets.IntText(
    value=0,
    description='Pre Days:',
    disabled=False
)

w_postDays = widgets.IntText(
    value=0,
    description='Post Days:',
    disabled=False
)

w_nsamples = widgets.IntText(
    value=1000,
    description='Number of Samples:',
    disabled=False
)

w_split_value = widgets.FloatText(
    value=0.8,
    description='Split Value:',
    disabled=False
)

w_accuracy = widgets.Checkbox(
    value=False,
    description='Accuracy Assessment',
    disabled=False
)

w_analysis_type = widgets.Dropdown(
    options=[
        ('Flood Mapping', 'flood_mapping'),
        ('Flood Mapping + Susceptibility', 'flood_susceptibility'),
        ('All (Flood Mapping + SUS + Exposure)', 'all')
    ],
    value='flood_mapping',
    description='Analysis Type:',
)

w_export = widgets.Checkbox(
    value=False,
    description='Export Results',
    disabled=False
)

w_run_button = widgets.Button(
    description='RUN',
    button_style='success',
    tooltip='Run the analysis'
)

# Output for logs
w_output = widgets.Output(layout={'border': '2px solid blue', 'height': '200px', 'overflow': 'auto'})

# Region selection widgets
region_options = [
    'Select an option',  # default to map bounds
    'Draw shapes on map',  # rectangle, polygon. If a point, use next option
    'Input point and buffer',  # input point coordinates and buffer distance
    'Rectangle from BBox',  # input bounding box coordinates
    'Upload GeoJSON',  # upload geometry in .geojson
    'Select ADM2 Name'  # select ADM2 name and process
]

w_region = widgets.Dropdown(
    options=region_options,
    description='Region:'
)

w_point = widgets.Text(
    description='Point (lat, lon):'
)

w_buffer = widgets.FloatText(
    description='Buffer (km):'
)

w_bbox = widgets.Text(
    description='BBox (xmin, ymin, xmax, ymax):'
)

w_geojson = widgets.Text(
    description='Link to GeoJSON file:'
)

w_adm2 = widgets.Text(
    description='ADM2 Name:'
)

w_region_detail = widgets.VBox()

# Function to update the region detail widget based on selection
def update_region_detail(*args):
    if w_region.value == 'Input point and buffer':
        w_region_detail.children = [w_point, w_buffer]
    elif w_region.value == 'Rectangle from BBox':
        w_region_detail.children = [w_bbox]
    elif w_region.value == 'Select ADM2 Name':
        w_region_detail.children = [w_adm2]
    else:
        w_region_detail.children = []

w_region.observe(update_region_detail, 'value')

# Display the region selection widgets
#display(case_study, w_region, w_region_detail)

# Function to set parameters based on case study selection
def set_case_study_params(change):
    if w_case_study.value == 'shikarpur':
        w_startDate.value = datetime.date(2022, 3, 1)
        w_endDate.value = datetime.date(2022, 8, 1)
        w_preDays.value = 60
        w_postDays.value = 30
        w_region.value = 'Select ADM2 Name'
        w_adm2.value = 'Shikarpur'
        
        
        
        # Set other parameters specific to Shikarpur
    elif w_case_study.value == 'nhamatanda':
        w_startDate.value = datetime.date(2019, 1, 1)
        w_endDate.value = datetime.date(2019, 3, 19)
        w_preDays.value = 60
        w_postDays.value = 1
        w_region.value = 'Select ADM2 Name'
        w_adm2.value = 'Nhamatanda'
        
        
        # Set other parameters specific to Nhamatanda
    elif w_case_study.value == 'ernukulam':
        w_startDate.value = datetime.date(2018, 3, 1)
        w_endDate.value = datetime.date(2018, 8, 7)
        w_preDays.value = 60
        w_postDays.value = 20
        w_region.value = 'Select ADM2 Name'
        w_adm2.value = 'Ernukulam'
        
        # Set other parameters specific to Ernukulam
    elif w_case_study.value == 'sylhet':
        w_startDate.value = datetime.date(2022, 1, 1)
        w_endDate.value = datetime.date(2022, 5, 17)
        w_preDays.value = 60
        w_postDays.value = 10
        w_region.value = 'Select ADM2 Name'
        w_adm2.value = 'Sylhet'

w_case_study.observe(set_case_study_params, names='value')

def getRegion():
    
    global city_shp
    
    city_shp = None    
    region = None
    
    if w_region.value == 'Draw shapes on map':
        print('Use geometry drawn on map')
        region = Map.user_roi

    elif w_region.value == 'Input point and buffer':
        coord = w_point.value.split(',')
        coord = [float(a) for a in coord[:2]]
        region = ee.Geometry.Point(coord).buffer(w_buffer.value * 1000)

    elif w_region.value == 'Rectangle from BBox':
        poly_coord = w_bbox.value.split(',')
        poly_coord = [float(a) for a in poly_coord]
        region = ee.Geometry.BBox(*poly_coord)

    elif w_region.value == 'Upload GeoJSON':
        url_geojson = w_geojson.value
        with open(os.path.abspath(url_geojson), encoding="utf-8") as f:
            geo_json = json.load(f)
        if geo_json["type"] == "FeatureCollection":
            region = ee.FeatureCollection(geo_json)
        elif geo_json["type"] == "Feature":
            region = ee.Geometry(geo_json['geometry'])

    elif w_region.value == 'Select an option':
        region = ee.Geometry.BBox(*Map.get_bounds())

    elif w_region.value == 'Select ADM2 Name':
        adm2_name = w_adm2.value
        city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
                    .filter(ee.Filter.eq('shapeName', adm2_name))
                    
        print(f"Selected ADM2 Name: {adm2_name}")
        
        bbox = city_shp.geometry().bounds()
        region = ee.Geometry.Polygon(bbox.coordinates().get(0))

    # if region not geometry convert to geometry
    if not isinstance(region, ee.Geometry):
        region = ee.Geometry(region)
        
    return region

def get_flood_layer():
    """
    Get flood layer
    """
    
    # Fetching pre and post-flood images
    s1_pre = get_s1_col(startDate, predays, aoi).select(['VV', 'VH'])
    s1_post = get_s1_col(endDate, postdays, aoi).select(['VV', 'VH'])

    print('Images in S1 Pre: ', s1_pre.size().getInfo())
    print('Images in S1 Post: ', s1_post.size().getInfo())

    # Calculate Z-score
    zscore = calculate_zscore(s1_pre, s1_post, aoi)

    # Generate flood masks
    flood_class, flood_layer = map_floods(zscore, aoi, zvv_value, zvh_value, water_value, elev_value, slope_value)

    print('Done with flood masking...')    


    flood_mapped = flood_mapping(aoi, s1_post, flood_layer, num_samples, split, city, export, accuracy)
    print('Done with flood mapping...')

    return flood_class, flood_mapped



# Define event handler
def on_run_button_clicked(b):
    with w_output:
        
        global aoi, num_samples, split, accuracy, analysis, export, startDate, endDate, predays, postdays, zvv_value, zvh_value, water_value, elev_value, slope_value, city
        w_output.clear_output()
        # Collect input values
        city = 'FMSE'
        aoi = getRegion()
        num_samples = w_nsamples.value
        split = w_split_value.value
        accuracy = w_accuracy.value
        analysis = w_analysis_type.value
        export = w_export.value
        
        
        startDate = ee.Date(w_startDate.value.strftime('%Y-%m-%d'))
        endDate = ee.Date(w_endDate.value.strftime('%Y-%m-%d'))
        predays = w_preDays.value
        postdays = w_postDays.value
        
        # Set default values
        zvv_value = -3
        zvh_value = -3
        water_value = 75
        elev_value = 900
        slope_value = 15
        
        print('Analysis started...')
        
        # Example function call (replace with actual FMSE function)
        flood_class, flood_mapped = get_flood_layer()

        
        if city_shp is not None:
            Map.add_basemap("SATELLITE")
            Map.centerObject(city_shp, 10)
            Map.addLayer(city_shp, {}, 'Selected City', False)
            Map.addLayer(flood_class.clip(city_shp), {'min': 0, 'max': 4, 'palette': ['#FFFFFF','#FFA500','#FFFF00','#FF0000','#0000FF']}, 'Flood class', False)
            Map.addLayer(flood_mapped.clip(city_shp), {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'Flood layer')

        else:
            # Display the resulting map
            Map.add_basemap("SATELLITE")
            Map.centerObject(aoi, 10)
            Map.addLayer(aoi, {}, 'AOI Boundary', False)
            Map.addLayer(flood_class, {'min': 0, 'max': 4, 'palette': ['#FFFFFF','#FFA500','#FFFF00','#FF0000','#0000FF']}, 'Flood class', False)
            Map.addLayer(flood_mapped, {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'Flood layer')


        # SUSCEPTIBILITY
        
        print('Starting susceptibility analysis...')
        
        flood_sus = susceptibility_analysis(aoi, endDate, flood_mapped, num_samples, split, city, export, accuracy)
        
        
        if city_shp is not None:
            Map.addLayer(flood_sus.clip(city_shp), {'min': 0.1, 'max': 0.9, 'palette': ['#1a9641', '#a6d96a', '#ffffbf', '#fdae61', '#d7191c']}, 'Flood Susceptibility')
        else:
            Map.addLayer(flood_sus, {'min': 0.1, 'max': 0.9, 'palette': ['#1a9641', '#a6d96a', '#ffffbf', '#fdae61', '#d7191c']}, 'Flood Susceptibility')

        
        print('Analysis completed...')
        
# Assign the event handler to the button
w_run_button.on_click(on_run_button_clicked)


def runApp():
    """
    Main app function
    """
    header = widgets.HTML("<h1 style='text-align: center'>FMSE: Flood Mapping Susceptibility and Exposure Assessment Tool</h1>")
    header2 = widgets.HTML("<p style='text-align: center'>Developed by: <a href='https://www.waleedgeo.com/'>Mirza Waleed</a></p>")
    
    left_sidebar = widgets.VBox([
        w_case_study,
        w_region,
        w_region_detail,
        w_startDate,
        w_endDate,
        w_preDays,
        w_postDays,
        w_nsamples,
        w_split_value,
        w_analysis_type,
        w_accuracy,
        w_export,
        w_run_button,
    ], layout=widgets.Layout(width='28%'))
    Map.add_basemap("SATELLITE")
    app = widgets.VBox([
        header,
        header2,
        widgets.HBox([
            left_sidebar,
            widgets.VBox([Map, w_output], layout=widgets.Layout(width='72%'))
        ])
    ])
    
    display(app)

runApp()




No such comm: 60d38d5ec15d4f108ed25369a14c4e0d
No such comm: c3d6fe1c85b94dacaa7ff93ba054a6a3
No such comm: 03d65cbcf1fa47febb8a5cda612db2e2
No such comm: 537eb21ee86849169d574aa97f9672ff
No such comm: 537eb21ee86849169d574aa97f9672ff
No such comm: 03d65cbcf1fa47febb8a5cda612db2e2
No such comm: 537eb21ee86849169d574aa97f9672ff


# ---

----

In [2]:


start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    disabled=False
)

pre_days = widgets.IntText(
    value=0,
    description='Pre Days:',
    disabled=False
)

post_days = widgets.IntText(
    value=0,
    description='Post Days:',
    disabled=False
)

num_samples = widgets.IntText(
    value=1000,
    description='Number of Samples:',
    disabled=False
)

split_value = widgets.FloatText(
    value=0.8,
    description='Split Value:',
    disabled=False
)

boundary_aoi = widgets.Checkbox(
    value=True,
    description='Boundary of AOI',
    disabled=False
)

analysis_type = widgets.Dropdown(
    options=[
        ('Flood Mapping', 'flood_mapping'),
        ('Flood Mapping + Susceptibility', 'flood_susceptibility'),
        ('All (Flood Mapping + SUS + Exposure)', 'all')
    ],
    value='flood_mapping',
    description='Analysis Type:',
)

save_results = widgets.Checkbox(
    value=False,
    description='Save Results',
    disabled=False
)

run_button = widgets.Button(
    description='RUN',
    button_style='success',
    tooltip='Run the analysis'
)

#output = widgets.Output()
w_btn_workflow_log = widgets.Output(layout={'border': '2px solid blue',
                                            'height': '200px',
                                            'overflow': 'auto'})
output = w_btn_workflow_log




region_options = [
        'Select an option',  # default to map bounds
        'Draw shapes on map',  # rectangle, polygon. If a point, use next option
        'Input point and buffer',  # input point coordinates and buffer distance
        'Rectangle from BBox',  # input bounding box coordinates
        'Upload GeoJSON',  # upload geometry in .geojson
        'Select ADM2 Name'  # select ADM2 name and process
    ]

# Widgets for detailed input based on region selection
w_region = widgets.Dropdown(
    options=region_options,
    description='Region:'
)
w_point = widgets.Text(
    description='Point (lat, lon):'
)
w_buffer = widgets.FloatText(
    description='Buffer (km):'
)
w_bbox = widgets.Text(
    description='BBox (xmin, ymin, xmax, ymax):'
)
w_adm2 = widgets.Text(
    description='ADM2 Name:'
)
w_region_detail = widgets.VBox()
# Function to update the region detail widget based on selection
def update_region_detail(*args):
    if w_region.value == 'Input point and buffer':
        w_region_detail.children = [w_point, w_buffer]
    elif w_region.value == 'Rectangle from BBox':
        w_region_detail.children = [w_bbox]
    elif w_region.value == 'Select ADM2 Name':
        w_region_detail.children = [w_adm2]
    else:
        w_region_detail.children = []
w_region.observe(update_region_detail, 'value')

def getRegion():
    region = None
    
    if w_region.value == 'Draw shapes on map':
        print('Use geometry drawn on map')
        region = Map.user_roi
    elif w_region.value == 'Input point and buffer':
        coord = w_point.value.split(',')
        coord = [float(a) for a in coord[:2]]
        region = ee.Geometry.Point(coord).buffer(w_buffer.value * 1000)
    elif w_region.value == 'Rectangle from BBox':
        poly_coord = w_bbox.value.split(',')
        poly_coord = [float(a) for a in poly_coord]
        region = ee.Geometry.BBox(*poly_coord)
    elif w_region.value == 'Upload GeoJSON':
        with open(os.path.abspath(region_json_path), encoding="utf-8") as f:
            geo_json = json.load(f)
        if geo_json["type"] == "FeatureCollection":
            region = ee.FeatureCollection(geo_json)
        elif geo_json["type"] == "Feature":
            region = ee.Geometry(geo_json['geometry'])
    elif w_region.value == 'Select an option':
        region = ee.Geometry.BBox(*Map.get_bounds())
    elif w_region.value == 'Select ADM2 Name':
        adm2_name = w_adm2.value
        city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
                    .filter(ee.Filter.eq('shapeName', adm2_name))
                    
        print(f"Selected ADM2 Name: {adm2_name}")
        
        bbox = city_shp.geometry().bounds()
        region = ee.Geometry.Polygon(bbox.coordinates().get(0))
    

    return region




def get_flood_layer():
    """
    Get flood layer
    """
    global flood_layer, aoi, city_shp
    
    city_shp = None
    city = 'Shikarpur'
    city_shp = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2')\
        .filter(ee.Filter.eq('shapeName', city))
    aoi = boundary(city_shp)
    # Shikarpur
    startDate = ee.Date('2022-03-01')
    endDate = ee.Date('2022-08-01')
    predays = 60
    postdays = 30
    
    
    
    # optional
    split = 0.8
    zvv_value = -3
    zvh_value = -3
    water_value = 75
    elev_value = 900
    slope_value = 15
    num_samples = 1000
    under_estimate = False

    # Fetching pre and post-flood images
    s1_pre = get_s1_col(startDate, predays, aoi).select(['VV', 'VH'])
    s1_post = get_s1_col(endDate, postdays, aoi).select(['VV', 'VH'])

    print('Images in S1 Pre: ', s1_pre.size().getInfo())
    print('Images in S1 Post: ', s1_post.size().getInfo())

    # Calculate Z-score
    zscore = calculate_zscore(s1_pre, s1_post, aoi)

    # Generate flood masks
    flood_class, flood_layer = map_floods(zscore, aoi, zvv_value, zvh_value, water_value, elev_value, slope_value, under_estimate)

    print('Done with flood masking...')    

    # Run flood mapping example
    flood_mapped = flood_mapping(aoi, s1_post, flood_layer, num_samples, split, city)
    print('Done with flood mapping...')

    
    return flood_class, flood_mapped





#  global region, start_date, end_date
#  global feature_names, bands, spectral_list, var_list
#  global split_ratio, seed, scale
#
#  seed = 42 # hard-coded
#  # scale: pixel resultion for sampling
#
#  start_date = w_start_date.value.strftime("%Y-%m-%d")
#  end_date = w_end_date.value.strftime("%Y-%m-%d")
#  # to-do: not valid op
#  if end_date < start_date:
#    raise Exception("End_date < start_date.")
#    return None
#
#  region = getRegion()
#
#  split_ratio = w_split_ratio.value
  
  

Map = geemap.Map()
# Define event handler
def on_run_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        #aoi = aoi_options.value
        #start = start_date.value
        #end = end_date.value
        #pre = pre_days.value
        #post = post_days.value
        #samples = num_samples.value
        #split = split_value.value
        #boundary = boundary_aoi.value
        #analysis = analysis_type.value
        #save = save_results.value
        
        # Example function call (replace with actual FMSE function)
        # result_map = run_analysis(aoi, start, end, pre, post, samples, split, boundary, analysis, save)
        
        # Display the resulting map
        # display(result_map)

        flood_class, flood_mapped = get_flood_layer()

        Map.addLayer(aoi, {}, 'AOI Boundary', False)
        Map.addLayer(flood_class.clip(city_shp), {'min': 0, 'max': 4, 'palette': ['#FFFFFF','#FFA500','#FFFF00','#FF0000','#0000FF']}, 'Flood class', False)# non-flooded, vv, vh, vv+vh, water
        Map.addLayer(flood_mapped.clip(city_shp), {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'Flood layer', False)
        #Map.addLayer(flood_binary, {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'Flood Binary')
        #Map.addLayer(flood_susceptibility_prob, {'min': 0.1, 'max': 0.9, 'palette': ['#1a9641', '#a6d96a', '#ffffbf', '#fdae61', '#d7191c']}, 'Flood Susceptibility')

        #Map.addLayer(sus_catagory, {'min': 1, 'max': 5, 'palette': ['#1a9641', '#a6d96a', '#ffffbf', '#fdae61', '#d7191c']}, 'Flood Susceptibility Categorical')
        
        
        # For now, display the inputs to check
        #print(f"AOI: {aoi}")
        #print(f"Start Date: {start}")
        #print(f"End Date: {end}")
        #print(f"Pre Days: {pre}")
        #print(f"Post Days: {post}")
        #print(f"Number of Samples: {samples}")
        #print(f"Split Value: {split}")
        #print(f"Boundary of AOI: {boundary}")
        #print(f"Analysis Type: {analysis}")
        #print(f"Save Results: {save}")
        
        # Placeholder for displaying geemap.Map
        #result_map = geemap.Map()
        #display(result_map)

# Assign the event handler to the button
run_button.on_click(on_run_button_clicked)

# Display the widgets




def runApp():
  """
  Main app function
  """

  header = widgets.HTML("<h1 style='text-align: center'>FMSE: Flood Mapping Susceptibility and Exposure Assessment Tool</h1>")
  header2 = widgets.HTML("<h3 style='text-align: center'>Developed by: <a href='https://www.waleedgeo.com/'>Mirza Waleed</a></h3>")
  left_sidebar=widgets.VBox([
    w_region,
    w_region_detail,
    start_date,
    end_date,
    pre_days,
    post_days,
    num_samples,
    split_value,
    boundary_aoi,
    analysis_type,
    save_results,
    run_button,
    ], layout=widgets.Layout(width='28%')
  )

  app = widgets.VBox([
      header,
      header2,
      widgets.HBox([left_sidebar,
                    widgets.VBox([Map, w_btn_workflow_log],
                                 layout=widgets.Layout(width='72%'))
                  ]),

      #bottom_tab,
  ])

  display(app)

runApp()

In [ ]:
widgets.VBox([
    aoi_options,
    start_date,
    end_date,
    pre_days,
    post_days,
    num_samples,
    split_value,
    boundary_aoi,
    analysis_type,
    save_results,
    run_button,
    output
])






region_options = [
    'Select an option',  # default to map bounds
    'Draw shapes on map',  # rectangle, polygon. If a point, use next option
    'Input point and buffer',  # input point coordinates and buffer distance
    'Rectangle from BBox',  # input bounding box coordinates
    'Upload GeoJSON',  # upload geometry in .geojson
    'Select ADM2 Name'  # select ADM2 name and process
]

# Widgets for detailed input based on region selection
w_region = widgets.Dropdown(
    options=region_options,
    description='Region:'
)

w_point = widgets.Text(
    description='Point (lat, lon):'
)
w_buffer = widgets.FloatText(
    description='Buffer (km):'
)
w_bbox = widgets.Text(
    description='BBox (xmin, ymin, xmax, ymax):'
)
w_adm2 = widgets.Text(
    description='ADM2 Name:'
)

w_region_detail = widgets.VBox()

# Function to update the region detail widget based on selection
def update_region_detail(*args):
    if w_region.value == 'Input point and buffer':
        w_region_detail.children = [w_point, w_buffer]
    elif w_region.value == 'Rectangle from BBox':
        w_region_detail.children = [w_bbox]
    elif w_region.value == 'Select ADM2 Name':
        w_region_detail.children = [w_adm2]
    else:
        w_region_detail.children = []

w_region.observe(update_region_detail, 'value')


def getRegion():
    region = None
    
    if w_region.value == 'Draw shapes on map':
        print('Use geometry drawn on map')
        region = Map.user_roi

    elif w_region.value == 'Input point and buffer':
        coord = w_point.value.split(',')
        coord = [float(a) for a in coord[:2]]
        region = ee.Geometry.Point(coord).buffer(w_buffer.value * 1000)

    elif w_region.value == 'Rectangle from BBox':
        poly_coord = w_bbox.value.split(',')
        poly_coord = [float(a) for a in poly_coord]
        region = ee.Geometry.BBox(*poly_coord)

    elif w_region.value == 'Upload GeoJSON':
        with open(os.path.abspath(region_json), encoding="utf-8") as f:
            geo_json = json.load(f)
        if geo_json["type"] == "FeatureCollection":
            region = ee.FeatureCollection(geo_json)
        elif geo_json["type"] == "Feature":
            region = ee.Geometry(geo_json['geometry'])

    elif w_region.value == 'Select an option':
        region = ee.Geometry.BBox(*Map.get_bounds())

    elif w_region.value == 'Select ADM2 Name':
        adm2_name = w_adm2.value
        # Here you can add your logic to get the region by ADM2 name
        # For example, if you have a dictionary or API that maps ADM2 names to geometries
        # region = get_region_by_adm2_name(adm2_name)
        print(f"Selected ADM2 Name: {adm2_name}")
        # Placeholder for actual implementation
        region = ee.Geometry.Point([0, 0])  # Replace with actual logic
        
    return region

